In [138]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [139]:
# Loading data

In [140]:
with open('admission_vs_ventilation.json' , 'rt') as INFILE:
    jsondata =  json.load(INFILE)


In [141]:
# wrangling data to form dataframe

In [142]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

def parse_date(datestring):
    '''Convert a date string into a pandas datetime object '''
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata):
    mydatalist =rawdata['data'] 
    dates=[dictionary['date'] for dictionary in mydatalist] 
    dates.sort()#ordered
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index = pd.date_range(startdate, enddate, freq='D')
    admission_vs_ventilationdf = pd.DataFrame(index=index, columns=['Ventilation','Hospital'])
    for entry in mydatalist:
        date=parse_date(entry['date'])
        for column in ['Ventilation' , 'Hospital']: 
            if pd.isna(admission_vs_ventilationdf.loc[date,column]):
                value=float(entry[column]) if entry[column]!= None else 0.0
                admission_vs_ventilationdf.loc[date, column] = value
    admission_vs_ventilationdf.fillna(0.0, inplace=True)
    return admission_vs_ventilationdf

df = pd.DataFrame(wrangle_data(jsondata))
df

,Ventilation,Hospital
2020-03-23,0.0,4880.0
2020-03-24,0.0,6598.0
2020-03-25,0.0,8683.0
2020-03-26,0.0,10612.0
2020-03-27,0.0,12841.0
...,...,...
2021-10-22,889.0,568261.0
2021-10-23,869.0,569155.0
2021-10-24,885.0,0.0
2021-10-25,913.0,0.0


In [143]:
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    return {} # return data read from the API


In [144]:
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='PANIC', # you may want to change this...
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Keep calm and carry on",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='danger', description='PANIC', icon='exclamation-triangle', style=ButtonStyle(), tooltip='…

In [156]:
def timeseries_graph(walk):
    df[walk].plot()
    plt.show() # important! update won't work properly without this
    
whichwalk=wdg.Dropdown(
    options=['Ventilation', 'Hospital'],
    value='Ventilation',
    description='Walk no: ',
    disabled=False
)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=whichwalk.value
    if current==whichwalk.options[0]:
        other=whichwalk.options[1]
    else:
        other=whichwalk.options[0]
    whichwalk.value=other # forces the redraw
    whichwalk.value=current # now we can change it back
    
    
graph=wdg.interactive_output(timeseries_graph, {'walk': whichwalk})
    
display(whichwalk, graph)

Dropdown(description='Walk no: ', options=('Ventilation', 'Hospital'), value='Ventilation')

Output()